# Guys Thesis notebook

## imports and helper function

In [1]:
# math
import numpy as np
import math
import astropy
from astropy.coordinates import SkyCoord
import astropy.units as ua

# oparating system and times and file managing
import os
import time
import pickle
import re

# better resource usage
import multiprocessing
# import mpi4py
# import numba as nb

# simulation file reading and analysis
import h5py 
import gizmo_analysis as gizmo
import halo_analysis as halo
# import utilities as ut
import yt
# from yt.extensions.astro_analysis.halo_analysis import HaloCatalog
import unyt as u
import guyProjectionPlot as gpp

# for converting stellar-mass to halo-mass
# from fire_an.mstar_mhalo import loader_smdpl_sfr as ldsmdpl
# from fire_an.makeplots.litcomp import obsdataread as ob


# visuals
from PIL import Image
import ipywidgets as wg
from matplotlib.colors import Normalize
from matplotlib.ticker import AutoLocator, MultipleLocator, MaxNLocator
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

#interpolation
from scipy.interpolate import griddata

# galaxy alignment and centering
import edens as ed

## Data loading and preconfoguration of a single simulations

In [18]:
# choosing the simulation to analyze
galaxies_list = ["m11b_res2100","m11d_res7100","m11e_res7100","m11h_res7100","m11i_res7100","m11q_res880",
                 "m12b_res7100","m12c_res7100","m12f_res7100","m12i_res7100","m12m_res7100","m12r_res7100","m12w_res7100","m12z_res4200"
                 ,"m12_elvis_RomeoJuliet_res3500","m12_elvis_ThelmaLouise_res4000","m12_elvis_RomulusRemus_res4000"]
simulation_galaxy = "m12i_res7100"
# simulation_galaxy = "m11d_res7100"
# galaxy = "m11b"
# galaxy = "m11e"
# galaxy = "m11h"
# galaxy = "m11i"
# galaxy = "m11q"
# galaxy = "m12m"
# galaxy = "m12z"
# galaxy = "m12b"
# galaxy = "m12c"
# galaxy = "m12f"
# simulation_galaxy = "m12_elvis_RomeoJuliet_res3500"
# galaxy = "m12_elvis_ThelmaLouise"
# galaxy = "m12r"
# galaxy = "m12w"
# simulation_galaxy = "m12z_res4200"
# simulation_galaxy = "m12_elvis_RomulusRemus_res4000"

# Split the string based on "_res"
parts = simulation_galaxy.split("_res")

# Take the first part (everything before "_res")
galaxy = parts[0]
simulation_directory = "Sims/" + simulation_galaxy + "/output"
# time_BC = 

snapshot_nums = [486,534,600] # simply corresponding to ~11.2Gyrs, ~12.4Gyrs and 13.7Gyrs
snapshot_num = snapshot_nums[2]


# particles types naming
gas = "PartType0"
star = "PartType4"
property = 'H_p0_number_density'

if galaxy == "m11q" or galaxy == "m12_elvis_RomeoJuliet" or galaxy == "m12_elvis_ThelmaLouise" or galaxy =="m12_elvis_RomulusRemus" or galaxy == "m12i" or galaxy == "m12b" or galaxy == "m12c" or galaxy == "m12f" or galaxy == "m12m" or galaxy == "m12r" or galaxy == "m12w" or galaxy == "m12z":
    directory_path = simulation_directory + "/snapdir_" + str(snapshot_num) + "/snapshot_" + str(snapshot_num) + ".0.hdf5"
elif galaxy == "m11b" or galaxy == "m11d" or galaxy == "m11e" or galaxy == "m11h" or galaxy == "m11i":
    directory_path = simulation_directory + "/snapshot_" + str(snapshot_num) + ".hdf5"


# some prefix
output_directory = "output/yt/"
ds = yt.load(directory_path)
ad = ds.all_data()
print(ds.current_time.in_units("Gyr"))

# loading simulation halos using halo_analysis
simulation_directory = "Sims/" + simulation_galaxy
redshift = round(ds.parameters['Redshift'],2)
print(redshift)
hal = halo.io.IO.read_catalogs('redshift', redshift, simulation_directory)
redshift = round(ds.parameters['Redshift'],1)
print(redshift)
# sp = ds.sphere("max", (100, "kpc")) 

yt : [INFO     ] 2024-01-04 15:17:12,622 ComovingIntegrationOn does not exist, falling back to OmegaLambda
yt : [INFO     ] 2024-01-04 15:17:12,639 Calculating time from 1.000e+00 to be 4.355e+17 seconds
yt : [INFO     ] 2024-01-04 15:17:12,640 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2024-01-04 15:17:12,696 Parameters: current_time              = 4.3545571088051405e+17 s
yt : [INFO     ] 2024-01-04 15:17:12,696 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-01-04 15:17:12,697 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-01-04 15:17:12,698 Parameters: domain_right_edge         = [60000. 60000. 60000.]
yt : [INFO     ] 2024-01-04 15:17:12,699 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2024-01-04 15:17:12,700 Parameters: current_redshift          = 0.0
yt : [INFO     ] 2024-01-04 15:17:12,701 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2024-01-04 15:17:12,701 Parameters: omega_m

13.79875880550213 Gyr
0.0

# in utilities.simulation.Snapshot():
* reading:  Sims/m12i_res7100/snapshot_times.txt

* input redshift = 0.0:  using snapshot index = 600, redshift = 0.000


# in halo_analysis.halo_io.IO():
* read 35838 halos from:  Sims/m12i_res7100/halo/rockstar_dm/catalog_hdf5/halo_600.hdf5

# in halo_analysis.halo_io.Particle():
* read 35838 halos, 55 have star particles, from:  Sims/m12i_res7100/halo/rockstar_dm/catalog_hdf5/star_600.hdf5
* assigning primary host and coordinates wrt it to halo catalog...  finished

0.0


## Data loading and preconfoguration of multiple simulations

In [2]:
%%time

# choosing the simulation to analyze
galaxies_list = ["m11b_res2100","m11d_res7100","m11e_res7100","m11h_res7100","m11i_res7100","m11q_res880",
                 "m12b_res7100","m12c_res7100","m12f_res7100","m12i_res7100","m12m_res7100","m12r_res7100","m12w_res7100","m12z_res4200"
                 ,"m12_elvis_RomeoJuliet_res3500","m12_elvis_ThelmaLouise_res4000","m12_elvis_RomulusRemus_res4000"]

# galaxies_list = ["m11b_res2100","m11d_res7100","m11e_res7100","m11h_res7100","m11i_res7100","m11q_res880",
#                  "m12b_res7100","m12c_res7100","m12i_res7100","m12r_res7100",
#                  "m12_elvis_RomeoJuliet_res3500"]
# still has no halo: m12m m12_elvis_RomeoJuliet_res3500 m12_elvis_ThelmaLouise_res4000
# galaxies_list = ["m12_elvis_RomeoJuliet_res3500","m12_elvis_ThelmaLouise_res4000","m12_elvis_RomulusRemus_res4000"]


# galaxies_list = ["m11b_res2100","m11d_res7100"]

# some prefix
output_directory = "output/yt/"
snapshot_nums = [486,534,600] # simply corresponding to ~11.2Gyrs, ~12.4Gyrs and 13.7Gyrs
# snapshot_nums = [600]
gas = "PartType0"
star = "PartType4"
directions = ["FaceOn","EdgeOn"]
# direction = ["EdgeOn"]
property = 'H_p0_number_density'
property = 'velocity'
property = 'density'
fraction_from_Rvir = 0.15

CPU times: user 13 µs, sys: 9 µs, total: 22 µs
Wall time: 37 µs


In [47]:
%%time
width = 0.2*main_halo_virial_radius
cell_size = width/1000

# width = 50*u.kpc
# cell_size = 0.1*u.kpc

num_cells = int(width/cell_size)
adj_width = cell_size*num_cells
print(L_disk_norm)
property = "velocity"
fields=('gas', property)
center = main_halo_center
arbitrary_vector = [1,1,3]
perpendicular_vector = np.cross(L_disk_norm,arbitrary_vector) # projection on Edge
print(perpendicular_vector)

fn = sp.save_as_dataset()
sp_ds = yt.load(fn)

print(sp_ds)


HI_proj_x = yt.ProjectionPlot(sp_ds.all_data(), perpendicular_vector, fields=fields,center = center, width= adj_width, buff_size = (num_cells,num_cells))
HI_proj_x.set_cmap(fields, cmap ='coolwarm')
HI_proj_x.set_log(fields, True)
HI_proj_x.set_zlim(fields, 1e8, 1e28)
HI_proj_x.save(str(perpendicular_vector2) + ".png")
HI_proj_x.show()

yt : [INFO     ] 2023-12-03 22:21:15,840 Saving field data to yt dataset: snapshot_600_sphere.h5.
yt : [INFO     ] 2023-12-03 22:21:15,905 Parameters: current_time              = 4.3545571088051405e+17 s
yt : [INFO     ] 2023-12-03 22:21:15,906 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-12-03 22:21:15,906 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2023-12-03 22:21:15,907 Parameters: domain_right_edge         = [60000. 60000. 60000.] code_length
yt : [INFO     ] 2023-12-03 22:21:15,908 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2023-12-03 22:21:15,909 Parameters: current_redshift          = 0.0
yt : [INFO     ] 2023-12-03 22:21:15,909 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2023-12-03 22:21:15,910 Parameters: omega_matter              = 0.272
yt : [INFO     ] 2023-12-03 22:21:15,911 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-12-03 22:21:15,911 Parameters: hubb

[-0.4863488214230095, -0.18168636310493685, -0.854666536938326]
[ 0.30960745  0.60437993 -0.30466246]


AttributeError: 'YTDataContainerDataset' object has no attribute 'properties'

## progress

In [24]:
directions = ["FaceOn","EdgeOn"]
m11_galaxies_list = ["m11b_res2100","m11d_res7100","m11e_res7100","m11h_res7100","m11i_res7100","m11q_res880"]
m12_galaxies_list = ["m12b_res7100","m12c_res7100","m12f_res7100","m12i_res7100","m12m_res7100","m12r_res7100","m12w_res7100","m12z_res4200"]
elvis_galaxies_list = ["m12_elvis_RomeoJuliet_res3500","m12_elvis_ThelmaLouise_res4000","m12_elvis_RomulusRemus_res4000"]
# elvis_galaxies_list = ["m12_elvis_RomeoJuliet_res3500"]
galaxies_matrix = [m11_galaxies_list,m12_galaxies_list,elvis_galaxies_list]
# galaxies_matrix = [elvis_galaxies_list]

fraction_from_Rvir = 0.3
property = 'H_p0_number_density'
# property = 'temperature'
succ = 0
total = 0
for direction in directions:
    for galaxies_list in galaxies_matrix:
        for simulation_galaxy in galaxies_list:
            galaxy = simulation_galaxy.split('_res')[0]
            if galaxy == "m12_elvis_RomeoJuliet" or galaxy == "m12_elvis_ThelmaLouise" or galaxy =="m12_elvis_RomulusRemus":
                for j in range(2): # since these have 2 main halos
                    halo_num = j+1
                    for i in range(len(snapshot_nums)):
                        snapshot_num = snapshot_nums[i]
                        try:
                            pixel_values = np.load(output_directory + galaxy + "/" + property + " pixel_values.npy",allow_pickle=True).item()[str(snapshot_num)][direction][j]
                            succ += 1
                            total += 1
                        except:
                            total += 1
                            print("didnt finished with " + property + " in galaxy " + galaxy + " with halo " + str(halo_num) + " in snapshot " + str(snapshot_num) + " in direction " + direction) 
            else:
                for i in range(len(snapshot_nums)):
                    snapshot_num = snapshot_nums[i]
                    try:
                        pixel_values = np.load(output_directory + galaxy + "/" + property + " pixel_values.npy",allow_pickle=True).item()[str(snapshot_num)][direction][0]
                        succ += 1
                        total += 1
                    except:
                        total += 1
                        print("didnt finished with " + property + " in galaxy " + galaxy + " in snapshot " + str(snapshot_num) + " in direction " + direction)
print("finished with " + str(round(succ/total,4)*100) + "% in " + property)

finished with 100.0% in H_p0_number_density


## update database

In [5]:
directions = ["FaceOn","EdgeOn"]
m11_galaxies_list = ["m11b_res2100","m11d_res7100","m11e_res7100","m11h_res7100","m11i_res7100","m11q_res880"]
m12_galaxies_list = ["m12b_res7100","m12c_res7100","m12f_res7100","m12i_res7100","m12m_res7100","m12r_res7100","m12w_res7100","m12z_res4200"]
# m12_galaxies_list = ["m12b_res7100"]
# elvis_galaxies_list = ["m12_elvis_RomeoJuliet_res3500","m12_elvis_ThelmaLouise_res4000","m12_elvis_RomulusRemus_res4000"]
# elvis_galaxies_list = ["m12_elvis_RomeoJuliet_res3500"]
# galaxies_matrix = [m11_galaxies_list,m12_galaxies_list,elvis_galaxies_list]
galaxies_matrix = [m11_galaxies_list,m12_galaxies_list]
# galaxies_matrix = [elvis_galaxies_list]

fraction_from_Rvir = 0.3
property = 'H_p0_number_density'
# property = 'temperature'
for direction in directions:
    for galaxies_list in galaxies_matrix:
        for simulation_galaxy in galaxies_list:
            galaxy = simulation_galaxy.split('_res')[0]
            # log_median_H_p0_column_density_R = np.load(output_directory + galaxy + "/log_median_" + property + ".npy",allow_pickle = True).item()
            # perc_log_H_p0_column_density_R = np.load(output_directory + galaxy + "/perc_" + property + ".npy",allow_pickle = True).item()
            # main_halo_stellar_mass = np.load(output_directory + galaxy + "/halo_stellar_masses.npy",allow_pickle = True).item()
            # R = np.load(output_directory + galaxy + "/Rvir.npy",allow_pickle = True).item()
            # main_halo_virial_radii = np.load(output_directory + galaxy + "/halo_virial_radii.npy",allow_pickle = True).item()
            # main_halo_centers = np.load(output_directory + galaxy + "/halo_centers.npy",allow_pickle=True).item()
            # L_disk_norms = np.load(output_directory + galaxy + "/L_disk_norms.npy",allow_pickle=True).item()
            property_pv = np.load(output_directory + galaxy + "/" + property + " pixel_values.npy",allow_pickle=True).item()
            
            for i in range(len(snapshot_nums)):
                snapshot_num = snapshot_nums[i]
                
                # log_median_H_p0_column_density_R[str(snapshot_num)] = log_median_H_p0_column_density_R[str(snapshot_num)][0]
                            
                # perc_log_H_p0_column_density_R[str(snapshot_num)] = perc_log_H_p0_column_density_R[str(snapshot_num)][0]
                           
                # main_halo_stellar_mass[str(snapshot_num)] = main_halo_stellar_mass[str(snapshot_num)][0]
                
                # R[str(snapshot_num)] = R[str(snapshot_num)][0]
                
                # main_halo_virial_radii[str(snapshot_num)] = main_halo_virial_radii[str(snapshot_num)][0]
                            
                # main_halo_centers[str(snapshot_num)] = main_halo_centers[str(snapshot_num)][0]    
                
                # L_disk_norms[str(snapshot_num)] = L_disk_norms[str(snapshot_num)][0]
    
                property_pv[str(snapshot_num)][direction][0] = property_pv[str(snapshot_num)][direction][0][0]
        
            # np.save(output_directory + galaxy + "/halo_centers.npy",main_halo_centers)
            # np.save(output_directory + galaxy + "/halo_virial_radii.npy",main_halo_virial_radii)
            # np.save(output_directory + galaxy + "/halo_stellar_masses.npy",main_halo_stellar_mass)
            # np.save(output_directory + galaxy + "/log_median_" + property + ".npy",log_median_H_p0_column_density_R)
            # np.save(output_directory + galaxy + "/perc_" + property + ".npy",perc_log_H_p0_column_density_R)
            # np.save(output_directory + galaxy + "/R.npy",R)
            # np.save(output_directory + galaxy + "/L_disk_norms.npy",L_disk_norms)
            # np.save(output_directory + galaxy + "/" + property + " pixel_values.npy",property_pv)
            
            print("finished with galaxy: ", galaxy)
            # print("main_halo_centers: ",main_halo_centers)
            # print("main_halo_virial_radii: ",main_halo_virial_radii)
            # print("main_halo_stellar_masses: ",main_halo_stellar_mass)
            # print("log_median_H_p0_column_density_R: ", log_median_H_p0_column_density_R)
            # print("perc_log_H_p0_column_density_R: ",perc_log_H_p0_column_density_R)
            # print("R :",R)
            # print("L_disk_norms: ",L_disk_norms)
            print("property: ",property_pv)

finished with galaxy:  m11b
property:  {'486': {'FaceOn': [array([1.34433445e+14, 1.34644028e+14, 1.34856984e+14, ...,
       6.26998063e+13, 6.27754175e+13, 6.28596140e+13])], 'EdgeOn': [array([[1.03378601e+14, 1.03557250e+14, 1.03734743e+14, ...,
        1.40345811e+14, 1.40420263e+14, 1.40494508e+14],
       [1.03561148e+14, 1.03744367e+14, 1.03926637e+14, ...,
        1.40624812e+14, 1.40698381e+14, 1.40771854e+14],
       [1.03740221e+14, 1.03928210e+14, 1.04115436e+14, ...,
        1.40907368e+14, 1.40979780e+14, 1.41052032e+14],
       ...,
       [1.63184783e+14, 1.63369331e+14, 1.63549865e+14, ...,
        1.20476068e+14, 1.20483274e+14, 1.20492127e+14],
       [1.62779125e+14, 1.62960762e+14, 1.63138217e+14, ...,
        1.20125315e+14, 1.20132427e+14, 1.20141009e+14],
       [1.62369564e+14, 1.62548981e+14, 1.62723691e+14, ...,
        1.19773922e+14, 1.19780966e+14, 1.19789243e+14]])]}, '534': {'FaceOn': [array([1.15752557e+14, 1.16052039e+14, 1.16349576e+14, ...,
       2.